In [ ]:
import os, sys
sys.path.append(os.path.abspath(".."))

import torch
from nanochat.gpt import GPT, GPTConfig

# 1. 定义配置 (这里使用一个小模型作为演示)
config = GPTConfig(
    vocab_size=50304, # 词表大小
    sequence_len=1024,# 最大上下文长度
    n_layer=4,        # Transformer 层数
    n_head=4,         # 注意力头数
    n_kv_head=4,   
    n_embd=128        # 嵌入维度
)

# 2. 实例化模型
model = GPT(config)

# 3. 初始化权重 (关键步骤)
# 应用特定的正态分布初始化，并将 lm_head 和部分投影层置零
model.init_weights()

print(f"模型参数量: {sum(p.numel() for p in model.parameters())}")

# --- 场景 A: 推理 (Inference) ---
# 构造模拟输入: Batch Size=2, Sequence Length=16
B, T = 2, 5
idx = torch.randint(0, config.vocab_size, (B, T))

# 前向传播 (不传 targets)
logits = model(idx)

# 输出形状: (Batch, Seq_Len, Vocab_Size)
print(f"\n输入形状: {idx.shape}")
print(f"Logits 形状: {logits.shape}") # -> torch.Size([2, 16, 50304])


# --- 场景 B: 训练 (Training) ---
# 构造模拟目标 (通常是 idx 向后移一位)
targets = torch.randint(0, config.vocab_size, (B, T))

# 前向传播 (传入 targets)
loss = model(idx, targets=targets)

# 输出是一个标量 Loss
print(f"\nTraining Loss: {loss.item():.4f}")

In [2]:
idx

tensor([[50225,  8774,  1447,  6861, 46644, 25897, 39148, 21490, 15152, 29623,
          1377, 18267, 20870, 17969, 33869, 40159],
        [41792, 36984, 17243, 16975, 12120, 11724, 27727, 35220, 29049, 11822,
          5502, 20497, 14346, 23403, 19113, 21961]])

In [ ]:
next_token = logits[:, -1, :]
print(f"next_token形状: {next_token.shape}")

In [8]:
import os, sys
sys.path.append(os.path.abspath(".."))

import torch
from nanochat.gpt import norm

x = torch.randn(2, 3, 4)   # (B=2, T=3, C=4)
y = norm(x)                # 仍是 (2,3,4)

y

tensor([[[-1.5229, -0.4442, -0.3876, -1.1546],
         [ 1.3176,  1.4261,  0.2118,  0.4307],
         [ 1.6044, -0.7211,  0.1985,  0.9308]],

        [[ 0.7581, -0.0231,  0.4089,  1.8048],
         [-1.1788, -1.2809,  0.6563,  0.7341],
         [ 0.5993, -1.7142, -0.2001,  0.8137]]])

In [9]:
y.size()

torch.Size([2, 3, 4])

In [10]:
import torch

M = torch.tensor([[1., 2., 3.],
                  [4., 5., 6.],
                  [7., 8., 9.]])      # 形状: (3, 3)

v = torch.tensor([10., 20., 30.])    # 形状: (3,)

# 变成列向量：shape (3, 1)
v_col = v.unsqueeze(1)

out = M + v_col
print(out)
print(out.shape)


tensor([[11., 12., 13.],
        [24., 25., 26.],
        [37., 38., 39.]])
torch.Size([3, 3])


In [12]:
v_col

tensor([[10.],
        [20.],
        [30.]])